### Import packages 

In [34]:
# Basic python packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import pickle

# Project specific modules 
import clean_data
import utils 
import calibration
import solution 
import make_shocks

import clean_data

In [35]:
# Run clean_data inline
clean_data.make_intermediate_pkls()

0


### Construct region list

- A region list is a dictionary:
  - Keys are region names
  - Values are lists of countries belonging to that region

- Our typical calibration includes:
  - Individual countries treated separately
  - EU countries aggregated together

- The utils.make_region_dict function:
  - Validates that the dictionary is properly constructed
  - Automatically builds "ROW" (Rest of World) as a list of remaining countries

- Our benchmark calibration has United States, Australia, Brazil, Canada, China, Great Britain, India, Indonesia, Japan, Mexico, South Korea, Russia, Turkey, and Taiwan as individual countries, plus the EU as an aggregate and a Rest of World (ROW) region

## Construct regions

In [36]:
regions = {}
regions['USA'] = ["USA"]

regions['Western Countries'] = ['AUS', 'AUT', 'BEL', 'CAN', 'CYP', 
                                'DEU', 'DNK', 'ESP', 'FIN', 'FRA',
                                'GBR', 'GRC', 'IRL', 'ITA', 'LUX',
                                'MLT', 'NLD', 'PRT', 'SWE']
regions['Eastern Europe'] = ['POL', "HUN", "SVN", "SVK", 'ROU', "CZE", "BGR", "EST", "LVA", "LTU"]


regions['East Asia']      = ['JPN', "KOR", 'TWN']

regions['RUS'] = ['RUS']
regions['CHN'] = ['CHN']


regions  = utils.make_region_dict(regions)



# Rename ROW to "Rest of the World" 


## Table 1: list of countries

In [37]:

# Create table showing region definitions
print("\\begin{table}[h]")
print("\\caption{Region Definitions}")
print("\\begin{center}")
print("\\begin{tabular}{ll}")
print("\\toprule")
print("Region & Countries \\\\")
print("\\midrule")

for region, countries in regions.items():
    # Format countries as comma-separated list
    if region != "ROW":
        country_list = ", ".join(countries)
        print(f"{region} & {country_list} \\\\")
    else:
        print("Rest of the World & Rest of the World \\\\")


print("\\bottomrule")
print("\\end{tabular}")
print("\\end{center}")
print("\\end{table}")

\begin{table}[h]
\caption{Region Definitions}
\begin{center}
\begin{tabular}{ll}
\toprule
Region & Countries \\
\midrule
USA & USA \\
Western Countries & AUS, AUT, BEL, CAN, CYP, DEU, DNK, ESP, FIN, FRA, GBR, GRC, IRL, ITA, LUX, MLT, NLD, PRT, SWE \\
Eastern Europe & POL, HUN, SVN, SVK, ROU, CZE, BGR, EST, LVA, LTU \\
East Asia & JPN, KOR, TWN \\
RUS & RUS \\
CHN & CHN \\
Rest of the World & Rest of the World \\
\bottomrule
\end{tabular}
\end{center}
\end{table}


### Steady-state calibration by cases 

In [38]:

# Calibrate steady-states (theta )

ss1 = calibration.calibrate_BGP(regions, dΩ_tilde_dp = True, theta =  1 + 1)
ss4 = calibration.calibrate_BGP(regions, dΩ_tilde_dp = True, theta =  4 + 1)
ss7 = calibration.calibrate_BGP(regions, dΩ_tilde_dp = True, theta =  7 + 1)

ss1_static = calibration.calibrate_BGP(regions, static = True, theta =  1 + 1)
ss4_static = calibration.calibrate_BGP(regions, static = True, theta =  4 + 1)
ss7_static = calibration.calibrate_BGP(regions, static = True, theta =  7 + 1)



Calibrate 
Passes sanity checks: True
Countries: ['USA', 'Western Countries', 'Eastern Europe', 'East Asia', 'RUS', 'CHN', 'ROW']
λ_GNE by country is:  [0.31444468 0.38380395 0.01975645 0.17544816 0.02511191 0.08337005
 0.22609311]
Average wedge on capital [2.37265394 2.48009535 2.17823181 2.16758689 2.63155583 1.33171217
 3.4222725 ]
Return to capital [0.106271   0.13750001 0.11211768 0.10385053 0.1335321  0.03759947
 0.20390641]
Calculate dΩ_tilde_dp...
Done!

Calibrate 
Passes sanity checks: True
Countries: ['USA', 'Western Countries', 'Eastern Europe', 'East Asia', 'RUS', 'CHN', 'ROW']
λ_GNE by country is:  [0.31444468 0.38380395 0.01975645 0.17544816 0.02511191 0.08337005
 0.22609311]
Average wedge on capital [2.37265394 2.48009535 2.17823181 2.16758689 2.63155583 1.33171217
 3.4222725 ]
Return to capital [0.106271   0.13750001 0.11211768 0.10385053 0.1335321  0.03759947
 0.20390641]
Calculate dΩ_tilde_dp...
Done!

Calibrate 
Passes sanity checks: True
Countries: ['USA', 'Western 

### Print calibration table (table 2 in paper)

In [39]:

importlib.reload(calibration)
out = calibration.get_calibration_objects(ss4, ['USA', 'Western Countries', 'Eastern Europe', 'RUS'])

# Create values DataFrame 
values_df = pd.DataFrame({
    'USA': [f"{out['USA']['r_i_bar']:.3f}", 
            f"{out['USA']['mu_bar']:.3f}",
            f"{out['USA']['inv_rate']:.3f}",
            f"{out['USA']['cap_income']:.3f}",
            f"{out['USA']['imports']:.3f}",
            f"{out['USA']['nfa']:.3f}"],
    'W. Countries': [f"{out['Western Countries']['r_i_bar']:.3f}",
            f"{out['Western Countries']['mu_bar']:.3f}",
            f"{out['Western Countries']['inv_rate']:.3f}", 
            f"{out['Western Countries']['cap_income']:.3f}",
            f"{out['Western Countries']['imports']:.3f}",
            f"{out['Western Countries']['nfa']:.3f}"],
    'E. Europe': [f"{out['Eastern Europe']['r_i_bar']:.3f}",
                     f"{out['Eastern Europe']['mu_bar']:.3f}",
                      f"{out['Eastern Europe']['inv_rate']:.3f}",
                      f"{out['Eastern Europe']['cap_income']:.3f}", 
                      f"{out['Eastern Europe']['imports']:.3f}",
                      f"{out['Eastern Europe']['nfa']:.3f}"],
    'RUS': [f"{out['RUS']['r_i_bar']:.3f}",
            f"{out['RUS']['mu_bar']:.3f}",
            f"{out['RUS']['inv_rate']:.3f}",
            f"{out['RUS']['cap_income']:.3f}",
            f"{out['RUS']['imports']:.3f}", 
            f"{out['RUS']['nfa']:.3f}"]
}, index=['Average return to capital',
          "Average as-if markup",
          'Investment rate',
          'Capital income',
          'Import share',
          'Net foreign assets'])

# Generate LaTeX table with note
latex_table = values_df.to_latex(escape=False)
latex_table = latex_table.replace('\\end{tabular}', 
                                '\\hline\n\\multicolumn{5}{l}{\\textit{Note:} Data averaged over 1995-2009.} \\\\\n\\end{tabular}')
print(latex_table)

\begin{tabular}{lllll}
\toprule
 & USA & W. Countries & E. Europe & RUS \\
\midrule
Average return to capital & 0.106 & 0.138 & 0.112 & 0.134 \\
Average as-if markup & 2.373 & 2.480 & 2.178 & 2.632 \\
Investment rate & 0.177 & 0.165 & 0.229 & 0.180 \\
Capital income & 0.420 & 0.409 & 0.498 & 0.474 \\
Import share & 0.121 & 0.148 & 0.408 & 0.213 \\
Net foreign assets & -0.188 & -0.140 & -0.436 & 0.021 \\
\bottomrule
\hline
\multicolumn{5}{l}{\textit{Note:} Data averaged over 1995-2009.} \\
\end{tabular}



## Make shocks

In [40]:
importlib.reload(make_shocks)
regions_west = ['USA'] + ['Western Countries'] + ['Eastern Europe'] + ['East Asia']


shock = {'dlogz' : make_shocks.tariff(ss5,
                   orig = regions_west,
                   dest = ['RUS'], 
                   scale = 3.0, bilateral = True)}


shock_static = {'dlogz' : make_shocks.tariff(ss5_static,
                   orig = regions_west,
                   dest = ['RUS'], 
                   scale = 3.0, bilateral = True)}

import nonlinear

### Run counterfactual (takes time, commented out as default)

In [41]:
# # # importlib.reload(solution)
vars = ['dlogC_c', 'dlogC', 'dlogK_c', 'dr', 'dS_c', 'dlogGNE_c', 'dlogK_c_ind', 'dlogp']

iter_no = 30

results_dynamic = {}

results_dynamic['theta1']  = nonlinear.solve(ss1, shock, iter = iter_no, vars = vars)
results_dynamic['theta4']  = nonlinear.solve(ss4, shock, iter = iter_no, vars = vars)
results_dynamic['theta7']  = nonlinear.solve(ss7, shock, iter = iter_no, vars = vars)


results_static = {}
results_static['theta1']  = nonlinear.solve(ss1_static, shock_static, iter = iter_no, vars = vars, full = False)
results_static['theta4']  = nonlinear.solve(ss4_static, shock_static, iter = iter_no, vars = vars, full = False)
results_static['theta7']  = nonlinear.solve(ss7_static, shock_static, iter = iter_no, vars = vars, full = False)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
Change in world consumption:  -0.011199045459393667
Change in country consumption:  [-0.00184223 -0.01055367 -0.06852916 -0.00383482 -0.33128106 -0.00035772
  0.00119637]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
Change in world consumption:  -0.0027583075772494373
Change in country consumption:  [-0.00033554 -0.00261678 -0.02026158 -0.00093748 -0.0886376   0.00024396
  0.001703  ]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
Change in world consumption:  -0.001454584691157496
Change in country consumption:  [-0.00013522 -0.00138399 -0.01177715 -0.00046489 -0.04955282  0.0001813
  0.00129029]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
Change in world consumption:  -0.004816835112700745
Change in country consumption:  [-0.00078847 -0.0053574  -0.0337999  -0.00155232 -0.25340022  0.00102852
  0.001509

## Construct Laspeyres index

In [46]:
C, N, K, F, K_c = ss4['C'], ss4['N'], ss4['K'], ss4['F'], ss4['K_c']
cap_idx   = range(C + N, C + N + K)
cap_idx_c = np.array([[i for i in cap_idx if ss5['C_map'][i] == c] for c in range(C)])


for name in results_dynamic:
    results_dynamic[name]['dvars_tot']['dlogK_c_l']  = np.zeros(C)
    for c in range(C):
        w_init    = ss4['λ'][cap_idx_c[c]] / np.sum(ss4['λ'][cap_idx_c[c]])
        deltaK    = results_dynamic[name]['dvars_tot']['dlogK_c_ind'][c]
        dlogK_c_l = np.sum(w_init * deltaK)                              
        
        results_dynamic[name]['dvars_tot']['dlogK_c_l'][c] = dlogK_c_l
    
print("Laspeyeres:", results_dynamic[name]['dvars_tot']['dlogK_c_l'])
print("Chained:", results_dynamic[name]['dvars_tot']['dlogK_c'])

Laspeyeres: [ 0.0001232  -0.00078405 -0.0134362  -0.00053382 -0.07701566  0.00133624
  0.00193635]
Chained: [ 0.00012543 -0.00073899 -0.01276748 -0.00052892 -0.06596368  0.00133724
  0.00195733]


## Output counterfactual results

In [49]:
def create_results_table(results):
    # Extract results and get indices for countries we want
    country_indices = [results['theta1']['ss_final']['countries'].index(c) for c in ["USA", "Western Countries", "Eastern Europe", "RUS", "CHN"]]

    # Create multi-level row headers
    countries = ['USA', 'Western Countries', 'Eastern Europe', 'Russia', 'China'] 
    thetas = ['$\\theta = 1$', '$\\theta = 4$', '$\\theta = 7$']
    rows = pd.MultiIndex.from_product([countries, thetas], names=['Country', 'Trade elasticity'])

    # Create columns
    columns = ['Consumption', 'GNE', 'Capital']

    # Initialize data array
    data = np.zeros((len(countries) * len(thetas), 3))

    # Fill in data for each country and theta combination
    for i, country in enumerate(countries):
        for j, theta in enumerate(['theta1', 'theta4', 'theta7']):
            idx = i * 3 + j
            data[idx, 0] = 100 * (np.exp(results[theta]['dvars_tot']['dlogC_c'][country_indices[i]]) - 1)
            data[idx, 1] = 100 * (np.exp(results[theta]['dvars_tot']['dlogGNE_c'][country_indices[i]]) - 1)
            data[idx, 2] = 100 * (np.exp(results[theta]['dvars_tot']['dlogK_c'][country_indices[i]]) - 1)

    # Create DataFrame
    df = pd.DataFrame(data, index=rows, columns=columns)

    # Convert to percentage changes and format to 2 decimal places with % symbol
    df = df.apply(lambda x: x.apply(lambda y: '{:.2f}\\%'.format(y)))

    # Generate LaTeX table
    latex_table = df.to_latex(escape=False)
    latex_table = latex_table.replace('[t]', '')
    latex_table = latex_table.replace('\\\\\\cline', '\\\\[3pt]\\cline')
    latex_table = "\\begin{table}\n" + latex_table + "\\end{table}"
    return latex_table

     
print(create_results_table(results_dynamic))



print("\n")

print(create_results_table(results_static))





\begin{table}
\begin{tabular}{lllll}
\toprule
 &  & Consumption & GNE & Capital \\
Country & Trade elasticity &  &  &  \\
\midrule
\multirow{3}{*}{USA} & $\theta = 1$ & -0.18\% & -0.19\% & -0.25\% \\
 & $\theta = 4$ & -0.03\% & -0.03\% & -0.02\% \\
 & $\theta = 7$ & -0.01\% & -0.01\% & 0.01\% \\
\cline{1-5}
\multirow{3}{*}{Western Countries} & $\theta = 1$ & -1.05\% & -1.06\% & -1.12\% \\
 & $\theta = 4$ & -0.26\% & -0.25\% & -0.21\% \\
 & $\theta = 7$ & -0.14\% & -0.12\% & -0.07\% \\
\cline{1-5}
\multirow{3}{*}{Eastern Europe} & $\theta = 1$ & -6.62\% & -6.68\% & -6.84\% \\
 & $\theta = 4$ & -2.01\% & -2.05\% & -2.13\% \\
 & $\theta = 7$ & -1.17\% & -1.21\% & -1.27\% \\
\cline{1-5}
\multirow{3}{*}{Russia} & $\theta = 1$ & -28.20\% & -28.89\% & -31.45\% \\
 & $\theta = 4$ & -8.48\% & -9.13\% & -10.37\% \\
 & $\theta = 7$ & -4.83\% & -5.45\% & -6.38\% \\
\cline{1-5}
\multirow{3}{*}{China} & $\theta = 1$ & -0.04\% & -0.00\% & 0.03\% \\
 & $\theta = 4$ & 0.02\% & 0.07\% & 0.14\% \\
 & $\t

## Generate wages and rental rates

In [44]:
# Dynamic model
C, N, K, F = ss5['C'], ss5['N'], ss5['K'], ss5['F']

F_idx   = range(C + N + K, C + N + K + F)
K_idx   = range(C + N , C + N + K)

F_idx_c = np.array(F_idx).reshape(C, F//C)
K_idx_c = np.array(K_idx).reshape(C, K//C)

dlogp = results_dynamic['theta4']['dvars_tot']['dlogp']

c = 4
Λ_f     = ss4['λ'][F_idx_c[c]] 

alpha         = ss4['λ'][K_idx_c[c]]  / np.sum(ss4['λ'][K_idx_c[c]])

dlogΛ_f = dlogp[F_idx_c[c]] 
dlogR   = dlogp[K_idx_c[c]]
real_wages_dyn =  np.sum(dlogΛ_f * Λ_f/np.sum(Λ_f)) -  dlogp[c]
real_rents_dyn =  np.sum(dlogR * alpha) -  dlogp[c] 

print("Dynamic model:")
print("Real wages:", np.exp(real_wages_dyn) - 1)
print("Real rents:", np.exp(real_rents_dyn) - 1)

# Static model
C, N, F = ss4_static['C'], ss4_static['N'], ss4_static['F']

F_idx_stat   = range(C + N, C + N + F)
F_idx_c_stat = np.array(F_idx_stat).reshape(C, F//C)

dlogp_stat = results_static['theta4']['dvars_tot']['dlogp']

Λ_f_stat     = ss4_static['λ'][F_idx_c_stat[c][1:]]

dlogΛ_f_stat = dlogp_stat[F_idx_c_stat[c][1:]]
dlogR_stat    = dlogp_stat[F_idx_c_stat[c][0]]

real_wages_stat =  np.sum(dlogΛ_f_stat * Λ_f_stat/np.sum(Λ_f_stat)) -  dlogp_stat[c]
real_rent_stat =  dlogR_stat -  dlogp_stat[c]


print("\nStatic model:")
print("Real wages:", np.exp(real_wages_stat) - 1)
print("Real rents:", np.exp(real_rent_stat) - 1)


Dynamic model:
Real wages: -0.09080496014647566
Real rents: 0.015473426243539201

Static model:
Real wages: -0.048322601029527124
Real rents: -0.046588334491318495
